# Paper figures

In [ ]:
%load_ext autoreload
%load_ext watermark

import cf_xarray.units
import dcpy
import eddydiff as ed
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

%aimport eddydiff
import xgcm
from datatree import DataTree

plt.style.use("ggplot")
xr.set_options(keep_attrs=True)

%watermark -iv

In [ ]:
plt.rcParams["figure.dpi"] = 140
plt.rcParams["figure.facecolor"] = "none"  # (0.91, 0.91, 0.91)
plt.rcParams["axes.facecolor"] = (0.99,) * 3
plt.rcParams.update(
    {
        "axes.spines.top": True,
        "axes.spines.bottom": True,
        "axes.spines.left": True,
        "axes.spines.right": True,
        "axes.edgecolor": (0.31,) * 3,
        "axes.grid.which": "both",
        "grid.linewidth": 0.5,
        "grid.alpha": 0.3,
        "grid.color": (0,) * 3,
    }
)

In [ ]:
mesomicrocolor = "darkorchid"
hirescolor = "darkorchid"
spinupcolor = "r"
spinupendcolor = "b"
lastcyclecolor = "deepskyblue"
color_ecco = "darkorange"


error_kwargs = dict(error="x", ls="none", marker=".")

In [ ]:
pop_natre = DataTree()

## Load data

### Isopycnal gradients from Argo climatology

Uses methodology of Cole et al (2015) on isopycnal temperature field from Roemmich & Gilson mapped fields

(Cole et al provide isopycnal salinity gradient estimates only).

In [ ]:
xr.open_dataset("datasets/cole-clim-gradient-natre.nc")

In [ ]:
# read, take mean in region; swap to pressure coordinate
argograd = (
    xr.open_dataset("datasets/cole-clim-gradient-natre.nc")
    .reset_coords("pres")
    .sel(lat=slice(24, 28), lon=slice(360 - 30, 360 - 25))
    .cf.guess_coord_axis()
)
argograd.lon.attrs["axis"] = "X"
argograd["lon"] = argograd.lon.cf.guess_coord_axis()
argo_delT2_plane = ed.plane_fit_gradient(argograd.CT).assign_coords(
    {"pres": argograd.pres.mean(["lon", "lat"])}
)
argo_delT2_plane.pres.attrs["positive"] = "down"

### Cole et al (2015)

See `dataset-processing.ipynb` for details

In [ ]:
cole = xr.open_dataset("datasets/cole-natre.nc")
cole_natre = cole[["diffusivity", "density_mean_depth", "depth_mean_sig"]].interp(
    lat=argograd.lat.data, lon=argograd.lon.data
)

Move from sigma to depth space since $K_e$ estimates are in depth space

In [ ]:
argo_dTiso = xgcm.transform.linear_interpolation(
    argograd.dTiso,
    argograd.pres,
    cole.pres,
    "sigma",
    "sigma",
    "pres",
)

In [ ]:
cole_natre["RediVar"] = cole_natre.diffusivity * argo_dTiso**2
cole_natre["RediVar"].attrs = {
    "long_name": "$K_e^{cole} |∇T_{iso}^{argo}|²$",
    "units": "°C²/s",
}
cole_natre = cole_natre.mean(["lat", "lon"])

### Groeskamp et al (2020)

In [ ]:
groeskamp = xr.open_dataset("datasets/groeskamp-natre.nc")
groeskamp["dTiso"] = argo_dTiso.interp(pres=groeskamp.pres)
groeskamp["eddy_var_0"] = groeskamp["Ke_0"] * groeskamp.dTiso**2
groeskamp["eddy_var"] = groeskamp["Ke"] * groeskamp.dTiso**2
groeskamp = groeskamp.mean(["lat", "lon"])

### NATRE 

In [ ]:
micro = xr.load_dataset("datasets/natre-density-binned.nc")
micro.pres.attrs["bounds"] = "pres_err"

# only one bin high up where there is not much eddy stirring signal.
micro["residual"] = np.abs(micro.residual)
micro["residual_err"] = np.abs(micro.residual_err)

### POP 1/10° 

#### Variance budget terms (Guo et al, 2022)

In [ ]:
pop_hires_natre = (
    xr.load_dataset("datasets/pop-110-variance-natre.nc")
    .pint.quantify()
    .pint.to("K**2/s")
)
pop_hires_natre

#### Climatological fields

In [ ]:
%load_ext autoreload
%autoreload

pop_hires_clim = (
    xr.load_dataset("datasets/pop-110-climatology-natre.nc")
    .pint.quantify()
    .squeeze("cycle")
    .map(dcpy.util.to_base_units)
)
pop_hires_clim["TEMP"].attrs["standard_name"] = "sea_water_potential_temperature"

# NOTE: This only adds delT2, and averages in box
pop_hires_clim_profile = ed.pop.calc_mean_redivar_profile(pop_hires_clim)


pop_hires_clim_profile

#### Combine

In [ ]:
pop_natre["hires/clim"] = DataTree(
    xr.merge([pop_hires_natre, pop_hires_clim_profile], join="exact")
)
pop_natre

### POP 1° 

#### spinup

In [ ]:
pop_natre["1deg/month1"] = DataTree(xr.open_dataset("datasets/pop-1-month1.nc"))
pop_natre["1deg/decadal"] = DataTree(xr.open_dataset("datasets/pop-1-spinup-natre.nc"))

### ECCO

In [ ]:
ecco = xr.open_dataset("datasets/ecco-climatology-natre.nc").set_coords("z_σ")

## Figure 3

In [ ]:
def plot_pop_natre_meso_micro(ax):
    (-1 * pop_natre["hires/clim"]["DISS"]).cf.plot(
        ax=ax, lw=1.5, color=mesomicrocolor, label="POP 1/10° HDIFF"
    )
    dcpy.plots.errorbar(
        micro,
        x="residual",
        y="pres",
        color=mesomicrocolor,
        label="FP2005: $⟨χ⟩/2 - K_ρ^m ∂_zθ_m^2$",
        ax=ax,
        **error_kwargs,
    )


f, ax = plt.subplot_mosaic(
    [["natre", "pop-hires", "params"]],
    sharey=False,
    sharex=True,
    constrained_layout=False,
)

### Mesoscale budget
pop_natre["hires/clim"]["BC"].cf.plot(
    ax=ax["pop-hires"], label="POP 1/10° $⟨u_e^h θ_e⟩.∇_hθ_m$"
)
np.abs(-1 * pop_natre["hires/clim"]["PKC"]).cf.plot(
    ax=ax["pop-hires"], label="POP 1/10° $-⟨w_eθ_e⟩.∂_zθ_m$"
)
# (-1 * pop_natre.VMIX).cf.plot(color="sienna", ax=ax[0], label="POP VMIX")
# (-1 * pop_natre.HDIFF).cf.plot(ax=ax[0], label="POP HDIFF", marker="x", ls="none")


#################################
######## Microscale budget: NATRE
#################################
dcpy.plots.fill_between_bounds(
    micro,
    "KρTz2",
    y="pres",
    color="k",
    label="FP2005: $K_ρ^m ∂_zθ_m^2$",
    ax=ax["natre"],
)

dcpy.plots.fill_between_bounds(
    micro,
    "chib2",
    y="pres",
    color="C0",
    label="FP2005: $⟨χ⟩/2$",
    ax=ax["natre"],
)

#################################
### Parameterizations
#################################
cole_natre.RediVar.plot(
    y="pres", label="$K_e^{cole} |∇_hT^{cole}|²$", color="k", ls="--", ax=ax["params"]
)

groeskamp.eddy_var.plot(
    y="pres",
    label="$K_e^{G2020} |∇_hT^{cole}|²$",
    color="limegreen",
    ls="--",
    ax=ax["params"],
)

#################################
### Cleanup
#################################
for axx in ax.values():
    plot_pop_natre_meso_micro(axx)
    axx.set_xscale("log")
    axx.set_xlabel("")
    axx.set_ylim([1850, 350])
    axx.set_xlim([1e-11, 1e-8])
    axx.xaxis.set_minor_locator(
        mpl.ticker.LogLocator(base=10, subs="all", numticks=120)
    )
    axx.legend(loc="upper center", bbox_to_anchor=(0.5, -0.15))
    axx.grid(True, which="both", lw=0.5)

    patch = mpl.patches.Rectangle(
        (0, 800), 1, 700, transform=axx.get_yaxis_transform(), alpha=0.1, zorder=1
    )
    axx.add_patch(patch)


ax["pop-hires"].set_xlabel("Rate of variance production or dissipation [°C²/s]")
ax["pop-hires"].set_title("Mesoscale budget: POP2 1/10°", fontsize="medium")
ax["natre"].set_title("Microscale budget: FP2005", fontsize="medium")
ax["params"].set_title("$K_e$ Parameterizations", fontsize="medium")


dcpy.plots.label_subplots(ax.values(), x=0.03, y=0.95)
dcpy.plots.clean_axes(list(ax.values()))
f.set_size_inches((8, 5.3))
f.savefig("images/figure3.pdf", bbox_inches="tight")
f.savefig("images/figure3.png", bbox_inches="tight")

## Figure 4

In [ ]:
f, axx = plt.subplots(1, 3, sharey=True, constrained_layout=True)
ax = dict(zip(["K", "delT2", "chi"], axx))

legend_handles = []

for pop_, color, label in zip(
    [
        pop_natre["hires/clim"],
        pop_natre["1deg/month1"],
        pop_natre["1deg/decadal"].isel(decade=5),
    ],
    [hirescolor, spinupcolor, spinupendcolor, lastcyclecolor],
    ["POP2 1/10°", "POP2 1° month 1", "POP2 1° decade 6"],
):
    kwargs = dict(
        edges=ed.pop.get_edges(pop_.ds), orientation="horizontal", lw=2, color=color
    )
    ax["delT2"].stairs(np.sqrt(pop_["delT2_plane"].data), **kwargs)
    if "KAPPA_ISOP" in pop_:
        legend_handles.append(ax["K"].stairs(pop_["KAPPA_ISOP"].to_numpy(), **kwargs))
        ax["chi"].stairs(pop_["RediVar"].to_numpy(), **kwargs, label=label)

# K
legend_handles.append(
    groeskamp.Ke.cf.plot(
        y="pres", ax=ax["K"], color="limegreen", ls="--", label="Groeskamp (2020)"
    )
)
legend_handles.append(
    cole_natre.diffusivity.cf.plot(
        y="pres", ax=ax["K"], color="k", ls="--", label="Cole (2015)"
    )
)
legend_handles.append(ecco.Ke.cf.plot(y="vertical", ax=ax["K"], color=color_ecco))


# ∇T2
for data, color, label in zip(
    [argo_delT2_plane, ecco.delT2_plane],
    ["k", color_ecco],
    ["Argo", "ECCO Adjusted"],
):
    if label == "MIMOC":
        continue
    np.sqrt(data).cf.plot(
        y="vertical", ax=ax["delT2"], color=color, label=label, _labels=False
    )

# χmeso
(-1 * pop_natre["hires/clim"]["DISS"]).cf.plot(
    ax=ax["chi"], lw=2, color=hirescolor, label="POP2 1/10°", _labels=False
)
(ecco.GM_CHI).sel(node="adj").cf.plot(
    ax=ax["chi"],
    y="vertical",
    lw=2,
    color=color_ecco,
    label=None,
    _labels=False,
)

dcpy.plots.errorbar(
    micro,
    x="residual",
    y="pres",
    color=mesomicrocolor,
    label="NATRE: $⟨χ⟩/2 - K_ρ^m ∂_zθ_m^2$",
    # lw=2.5,
    ax=ax["chi"],
    **error_kwargs,
)

ax["K"].set_title("")
ax["K"].set_ylim([3000, 200])
ax["K"].set_xlim([0, 2000])
ax["delT2"].set_xlim([1e-8, 4e-6])
ax["chi"].set_xlim([1e-12, 1e-8])
dcpy.plots.clean_axes(axx)

for a in axx:
    a.xaxis.tick_top()
    a.xaxis.set_label_position("top")
    patch = mpl.patches.Rectangle(
        (0, 800), 1, 700, transform=a.get_yaxis_transform(), alpha=0.1, zorder=1
    )
    a.add_patch(patch)

ax["K"].set_xlabel("(a) $K_{e}$ [m$^2$/s]", labelpad=12)
ax["delT2"].set_xlabel("(b) $|∇_ρ^hT|$ [C/m]", labelpad=12)
ax["chi"].set_xlabel("(c) $χ_{e} = K_{e} |∇_ρ^hT|^2$ [C$^2$/s]", labelpad=12)

ax["K"].set_ylabel("depth [m]")
ax["chi"].set_xscale("log")
ax["chi"].xaxis.grid(True, which="both")
f.legend(loc="outside right")

f.set_size_inches((9, 6))
f.savefig("images/figure4.pdf")
f.savefig("images/figure4.png")